### **5.4 - [Práctica] Group tasks with SubDAGs!**

¡En esta actividad, vas a agrupar las tasks transforms con un subdag!

<center><img src="https://i.postimg.cc/TwyGGhYt/a430.png"></center>
<center><img src="https://i.postimg.cc/RFjm09W4/a431.png"></center>

1.	Did you implement subdag_transforms.py?

Under dags/subdags/subdag_transforms.py

<center><img src="https://i.postimg.cc/J0ZLDgXJ/a432.png"></center>

In [ ]:
from airflow import DAG
from airflow.operators.bash import BashOperator

def subdag_transforms(parent_dag_id, child_dag_id, args):

    with DAG(f"{parent_dag_id}.{child_dag_id}",
        start_date=args['start_date'],
        schedule_interval=args['schedule_interval'],
        catchup=args['catchup']) as dag:

        transform_a = BashOperator(
            task_id='transform_a',
            bash_command='sleep 10'
        )

        transform_b = BashOperator(
            task_id='transform_b',
            bash_command='sleep 10'
        )

        transform_c = BashOperator(
            task_id='transform_c',
            bash_command='sleep 10'
        )
        return dag

2.	Did you implement the SubDagOperator in group_dag.py?

In group_dag.py

In [ ]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.subdag import SubDagOperator
from subdags.subdag_downloads import subdag_downloads
from subdags.subdag_transforms import subdag_transforms

from datetime import datetime

with DAG('group_dag', start_date=datetime(2022, 1, 1), 
    schedule_interval='@daily', catchup=False) as dag:

    args = {'start_date': dag.start_date, 'schedule_interval': dag.schedule_interval, 'catchup': dag.catchup}

    downloads = SubDagOperator(
        task_id='downloads',
        subdag=subdag_downloads(dag.dag_id, 'downloads', args)
    )

    check_files = BashOperator(
        task_id='check_files',
        bash_command='sleep 10'
    )

    transforms = SubDagOperator(
        task_id='transforms',
        subdag=subdag_transforms(dag.dag_id, 'transforms', args)
    )

    downloads >> check_files >> transforms